# Part 1 

In [1]:
from intcode import IntCodeComputer

all tests passed


In [2]:
program = list(map(int, open('data/input23').read().strip().split(',')))

We need to parse messages. Let's just go through the list and get a dictionary of messages that we then can dispatch.

In [3]:
from collections import defaultdict

def parse_messages(clear=True):

    messages = defaultdict(list)

    for c in range(50):
        prints = computers[c].state[-1]
        if len(prints) > 0:
            assert len(prints) % 3 == 0 
            for destination, x, y in zip(prints[::3], prints[1::3], prints[2::3]):
                messages[destination].append((x, y))
        
        if clear: 
            computers[c].state = computers[c].state[:-1] + ([],)
            
    return messages

Now let's try and put it together, removing messages once they're parsed and implementing a check for address 255.

In [4]:
# create computers
computers = {c: IntCodeComputer(program, resume=True) for c in range(50)}

# assign network address and run
for c in range(50):
    computers[c].run(input_values=[c])
    
while True:
    messages = parse_messages(clear=True)
    if 255 in messages:
        print(f"solution for part1: {messages[255][0][1]}")
        break
    for destination in range(50):
        if destination in messages:
            for message in messages[destination]:
                computers[destination].run(input_values=list(message))
        else:
            computers[destination].run(input_values=[-1])

solution for part1: 24922


# Part 2 

Let's create a NAT class and insert it at address 255 in the list of computers.

In [5]:
class NAT:
    def __init__(self):
        self.packet = [None, None]
        self.has_received = False
        
    def run(self, input_values):
        if len(input_values) > 0:
            assert len(input_values) % 2 == 0
            for x, y in zip(input_values[::2], input_values[1::2]):
                self.packet = [x, y]
                self.has_received = True

In [6]:
# create computers
computers = {c: IntCodeComputer(program, resume=True) for c in range(50)}
computers[255] = NAT()
nat_packets = []

# assign network address and run
for c in range(50):
    computers[c].run(input_values=[c])


while True:
    messages = parse_messages(clear=True)
    for destination in list(range(50)) + [255]:
        if destination in messages:
            for message in messages[destination]:
                computers[destination].run(input_values=list(message))
        else:
            if destination != 255:
                computers[destination].run(input_values=[-1])
    
    if len(messages) == 0:
        if computers[255].has_received:
            nat_packet = computers[255].packet
            nat_packets.append(nat_packet[1])
            if len(nat_packets) > 1:
                if nat_packets[-2:] == nat_packets[-1:] * 2:
                    print(f'solution for part2: {nat_packets[-1]}')
                    break
            computers[0].run(input_values=nat_packet)

solution for part2: 19478
